# Advanced Indexing

## Learning Objectives

* Orthogonal vs. Pointwise Indexing

## Overview

In the previous notebooks, we learned basic forms of indexing with Xarray (positional and name based dimensions, integer and label based indexing), datetime Indexing, and nearest neighbor lookups. Xarray positional indexing deviates from the NumPy when indexing with multiple arrays like `arr[[0, 1], [0, 1]]`.

In this tutorial we learn about this difference and how to do vectorized/pointwise indexing using Xarray.

For this notebook, first, we should learn about orthogonal (i.e. outer) and vectorized (i.e. pointwise) indexing concepts. 

* *Orthogonal* or *outer* indexing allows for indexing along each dimension independently, treating the indexers as one-dimensional arrays. The principle of outer or orthogonal indexing is that the result mirrors the effect of independently indexing along each dimension with integer or boolean arrays, treating both the indexed and indexing arrays as one-dimensional. This method of indexing is analogous to vector indexing in programming languages like MATLAB, Fortran, and R, where each indexer component *independently* selects along its corresponding dimension. This is the default behavior in Xarray.

* *Vectorized* indexing is a more general form of indexing that allows for arbitrary combinations of indexing arrays. This method of indexing is analogous to the broadcasting rules in NumPy, where the dimensions of the indexers are aligned and the result is determined by the shape of the indexers. This is the default behavior in NumPy.  


We can better understand this with an example: 

In [1]:
import numpy as np

# Create a 5x5 array with values from 1 to 25
np_array = np.arange(1, 26).reshape(5, 5)
np_array

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

Now create a Xarray DataArray from this NumPy array: 

In [54]:
import xarray as xr
da = xr.DataArray(np_array, dims=["x", "y"])
da

<xarray.DataArray (x: 5, y: 5)>
array([[ 1,  2, ...,  4,  5],
       [ 6,  7, ...,  9, 10],
       ...,
       [16, 17, ..., 19, 20],
       [21, 22, ..., 24, 25]])
Dimensions without coordinates: x, y

In [3]:
np_array [[0,2,4],[0,2,4]]

array([ 1, 13, 25])

In [4]:
da [[0,2,4],[0,2,4]]

<xarray.DataArray (dim_0: 3, dim_1: 3)>
array([[ 1,  3,  5],
       [11, 13, 15],
       [21, 23, 25]])
Dimensions without coordinates: dim_0, dim_1

The image below summarizes the difference between orthogonal and vectorized indexing for a 2D 5x5 array. 

![Orthogonal vs. Vectorized Indexing](../../images/orthogonal_vs_vectorized.png)

 Point-wise indexing, shown on the left, selects specific elements at given coordinates, resulting in an array of those individual elements. In the example shown, the indices `[0, 2, 4]`, `[0, 2, 4]` select the elements at positions (0, 0), (2, 2), and (4, 4), resulting in the values `[1, 13, 25]`. This is shown in NumPy indexing example. 


 In contrast, **orthogonal indexing** uses the same indices to select entire rows and columns, forming a cross-product of the specified indices. This method results in subarrays that include all combinations of the selected rows and columns. The example demonstrates this by selecting rows 0, 2, and 4 and columns 0, 2, and 4, resulting in a subarray containing `[[1, 3, 5], [11, 13, 15], [21, 23, 25]]`. This is shown in Xarray indexing example.
 
  The output of orthogonal indexing is a 3x3 array, while the output of vectorized indexing is a 1D array.

## Orthogonal Indexing in Xarray

If you only provide integers, slices, or unlabeled arrays (array without dimension names, such as `np.ndarray`, `list`, but not `DataArray()`) indexing can be understood as orthogonally (i.e. along independent axes, instead of using NumPy’s broadcasting rules to vectorize indexers). In the example above we saw this behavior, but let's see it in action with a real dataset.

In [5]:
import numpy as np
import pandas as pd
import xarray as xr


xr.set_options(display_expand_attrs=False)
np.set_printoptions(threshold=10, edgeitems=2)

ds = xr.tutorial.load_dataset("air_temperature")
da_air = ds.air
ds


<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes: (5)

In [6]:
selected_da = da_air.isel(time=0, lat=[2, 4, 10, 13], lon=[1, 6, 7])  # -- orthogonal indexing
selected_da

<xarray.DataArray 'air' (lat: 4, lon: 3)>
array([[249.79999, 243.09999, 242.39   ],
       [274.29   , 271.79   , 270.6    ],
       [277.4    , 280.6    , 280.9    ],
       [283.19998, 285.5    , 285.9    ]], dtype=float32)
Coordinates:
  * lat      (lat) float32 70.0 65.0 50.0 42.5
  * lon      (lon) float32 202.5 215.0 217.5
    time     datetime64[ns] 2013-01-01
Attributes: (11)

👆 please notice how the ouput if the indexing example above resulted in an array of 3x4. 

For more flexibility, you can supply `DataArray()` objects as indexers. Dimensions on resultant arrays are given by the ordered union of the indexers’ dimensions.

For example, in the example below we do orthogonal indexing using `DataArray()` objects. 

In [7]:
target_lat = xr.DataArray([31, 41, 42, 42], dims="degrees_north")
target_lon = xr.DataArray([200, 201, 202, 205], dims="degrees_east")

da_air.sel(lat=target_lat, lon=target_lon, method="nearest")  # -- orthogonal indexing

<xarray.DataArray 'air' (time: 2920, degrees_north: 4, degrees_east: 4)>
array([[[293.1    , 293.1    , 293.29   , 293.29   ],
        [284.6    , 284.6    , 284.9    , 284.19998],
        [282.79   , 282.79   , 283.19998, 282.6    ],
        [282.79   , 282.79   , 283.19998, 282.6    ]],

       [[293.19998, 293.19998, 293.9    , 294.19998],
        [283.29   , 283.29   , 285.19998, 285.19998],
        [281.4    , 281.4    , 282.79   , 283.5    ],
        [281.4    , 281.4    , 282.79   , 283.5    ]],

       ...,

       [[288.29   , 288.29   , 289.19   , 290.79   ],
        [282.09   , 282.09   , 281.59   , 282.38998],
        [280.99   , 280.99   , 280.38998, 280.59   ],
        [280.99   , 280.99   , 280.38998, 280.59   ]],

       [[289.49   , 289.49   , 290.38998, 291.59   ],
        [282.09   , 282.09   , 281.99   , 283.09   ],
        [281.38998, 281.38998, 280.59   , 280.99   ],
        [281.38998, 281.38998, 280.59   , 280.99   ]]], dtype=float32)
Coordinates:
    lat      (degrees_north) float32 30.0 40.0 42.5 42.5
    lon      (degrees_east) float32 200.0 200.0 202.5 205.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Dimensions without coordinates: degrees_north, degrees_east
Attributes: (11)

In the above example, you can see how the output shape is `time` x `lats` x `lons`. 


But what if we would like to find the information from the nearest grid cell to a collection of specified points (for example, weather stations or tower data)?

## Vectorized or Pointwise Indexing in Xarray

Like NumPy and pandas, Xarray supports indexing many array elements at once in a *vectorized* manner. 

**Vectorized indexing** or **Pointwise Indexing** using `DataArrays()` can be used to extract information from the nearest grid cells of interest, for example, the nearest climate model grid cells to a collection of specified weather station latitudes and longitudes.

```{hint}
To trigger vectorized indexing behavior, you will need to provide the selection dimensions with a different name than the original dimensions. This dimension name will be used in the output array.
```

In the example below, the selections of the closest latitude and longitude are renamed to an output dimension named `points`:

In [44]:
# Define target latitude and longitude (where weather stations might be)
lat_points = xr.DataArray([31, 41, 42, 42], dims="points")
lat_points

<xarray.DataArray (points: 4)>
array([31, 41, 42, 42])
Dimensions without coordinates: points

In [45]:
lon_points = xr.DataArray([200, 201, 202, 205], dims="points")
lon_points

<xarray.DataArray (points: 4)>
array([200, 201, 202, 205])
Dimensions without coordinates: points

Now, retrieve data at the grid cells nearest to the target latitudes and longitudes (weather stations):

In [23]:
da_air.sel(lat=lat_points, lon=lon_points, method="nearest")

<xarray.DataArray 'air' (time: 2920, points: 4)>
array([[293.1    , 284.6    , 283.19998, 282.6    ],
       [293.19998, 283.29   , 282.79   , 283.5    ],
       ...,
       [288.29   , 282.09   , 280.38998, 280.59   ],
       [289.49   , 282.09   , 280.59   , 280.99   ]], dtype=float32)
Coordinates:
    lat      (points) float32 30.0 40.0 42.5 42.5
    lon      (points) float32 200.0 200.0 202.5 205.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Dimensions without coordinates: points
Attributes: (11)

👆 Please notice how the shape of our `DataArray` is `time` x `points`, extracting time series for each weather stations. 


In [24]:
da_air.sel(lat=lat_points, lon=lon_points, method="nearest").dims

('time', 'points')

```{attention}
Please note that slices or sequences/arrays without named-dimensions are treated as if they have the same dimension which is indexed along.
```

For example:

In [43]:
da_air.sel(lat=[20, 30, 40], lon=lon_points, method="nearest")

<xarray.DataArray 'air' (time: 2920, lat: 3, points: 4)>
array([[[296.6    , 296.6    , 296.19998, 296.4    ],
        [293.1    , 293.1    , 293.29   , 293.29   ],
        [284.6    , 284.6    , 284.9    , 284.19998]],

       [[296.4    , 296.4    , 295.9    , 296.19998],
        [293.19998, 293.19998, 293.9    , 294.19998],
        [283.29   , 283.29   , 285.19998, 285.19998]],

       ...,

       [[293.69   , 293.69   , 293.88998, 295.38998],
        [288.29   , 288.29   , 289.19   , 290.79   ],
        [282.09   , 282.09   , 281.59   , 282.38998]],

       [[293.79   , 293.79   , 293.69   , 295.09   ],
        [289.49   , 289.49   , 290.38998, 291.59   ],
        [282.09   , 282.09   , 281.99   , 283.09   ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 20.0 30.0 40.0
    lon      (points) float32 200.0 200.0 202.5 205.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Dimensions without coordinates: points
Attributes: (11)

## Excersises

```{exercise}
:label: indexing-1

In the simple 2D 5x5 Xarray data array above, select the sub-array containing (0,0),(2,2),(4,4) : 
```



````{solution} indexing-1
:class: dropdown
```python

indices = np.array([0, 2, 4])

xs_da = xr.DataArray(indices, dims="points")
ys_da = xr.DataArray(indices, dims="points")

subset_da = da.sel(x=xs_da, y=xs_da)
subset_da
```
````


## Additional Resources

- [Xarray Docs - Indexing and Selecting Data](https://docs.xarray.dev/en/stable/indexing.html)
